In [77]:
import pandas as pd
chunksize = 1000

authors_df = pd.read_csv('data/authors.csv', chunksize=chunksize)
cited_by_count_df = pd.read_csv('data/cited_by_count.csv', chunksize=chunksize)
display_name_df = pd.read_csv('data/display_name.csv', chunksize=chunksize)
predicted_gender_df = pd.read_csv(
    'data/predicted_gender.csv', chunksize=chunksize)
works_df = pd.read_csv('data/works.csv', chunksize=chunksize)
works_api_url = pd.read_csv('data/works_api_url.csv', chunksize=chunksize)
x_concepts_df = pd.read_csv('data/x_concepts.csv', chunksize=chunksize)
year_df = pd.read_csv('data/year.csv', chunksize=chunksize)

In [5]:
def strip_url(chunk):
    # Process the chunk here
    # For example, you can print the first few rows of each chunk
    chunk['id'] = chunk['id'].apply(lambda x: str(x).split('/')[-1])
    # You can also perform other operations like filtering, aggregating, etc.
    # For demonstration, let's just return the first few rows
    return chunk


def fix_author_id(chunk):
    chunk.rename(columns={'id': 'author_id'}, inplace=True)
    return chunk


def fix_work_id(chunk):
    chunk.rename(columns={'id': 'work_id'}, inplace=True)
    return chunk
# List of DataFrames (or iterators of chunks)


dataframes = {
    # 'authors.csv': authors_df,
    # 'cited_by_count.csv': cited_by_count_df,
    # 'display_name.csv': display_name_df,
    # 'predicted_gender.csv': predicted_gender_df,
    # 'works_api_url.csv': works_api_url,
    # 'x_concepts.csv': x_concepts_df,
    'year.csv': year_df
}

for file_name, df in dataframes.items():
    # Initialize a flag to write headers
    write_header = True
    for chunk in df:
        # Process each chunk
        chunk = strip_url(chunk)
        chunk = fix_author_id(chunk)

        # Save the processed chunk to a new CSV file
        chunk.to_csv(f'data/processed_{file_name}',
                     mode='a', index=False, header=write_header)

        # After the first chunk, set the flag to False to skip headers
        write_header = False

In [6]:
columns = ['id', 'display_name', 'publication_date',
           'open_access', 'authorships', 'cited_by_count', 'keywords']
write_header = True
for chunk in works_df:
    chunk = chunk[columns]
    # Process each chunk
    chunk = strip_url(chunk)
    chunk = fix_work_id(chunk)

    # Save the processed chunk to a new CSV file
    chunk.to_csv('data/processed_works.csv',
                 mode='a', index=False, header=write_header)
    write_header = False

In [8]:
import ast

works_df = pd.read_csv('data/works.csv', chunksize=chunksize)
write_header = True
for chunk in works_df:
    chunk['authorships'] = chunk['authorships'].apply(ast.literal_eval)
    au_chunk = chunk[['work_id', 'authorships']].explode('authorships')
    au_chunk['author_id'] = au_chunk['authorships'].apply(
        lambda x: x['author']['id'] if x['author'] is not None else None)
    au_chunk['author_id'] = au_chunk['author_id'].apply(
        lambda x: str(x).split('/')[-1] if x is not None else None)
    au_chunk['author_display_name'] = au_chunk['authorships'].apply(
        lambda x: x['author']['display_name'] if x['author'] is not None else None)
    au_chunk.drop(columns=['authorships'], inplace=True)
    print(f"Processing chunk with {len(au_chunk)} rows")
    au_chunk.to_csv('data/processed_authorships.csv',
                    mode='a', index=False, header=write_header)
    write_header = False

Processing chunk with 5948 rows
Processing chunk with 5594 rows
Processing chunk with 5601 rows
Processing chunk with 6277 rows
Processing chunk with 6028 rows
Processing chunk with 6322 rows
Processing chunk with 6489 rows
Processing chunk with 5537 rows
Processing chunk with 5244 rows
Processing chunk with 6441 rows
Processing chunk with 6117 rows
Processing chunk with 6371 rows
Processing chunk with 5811 rows
Processing chunk with 5359 rows
Processing chunk with 5780 rows
Processing chunk with 5764 rows
Processing chunk with 5330 rows
Processing chunk with 5875 rows
Processing chunk with 5861 rows
Processing chunk with 7105 rows
Processing chunk with 5008 rows
Processing chunk with 7508 rows
Processing chunk with 5819 rows
Processing chunk with 6164 rows
Processing chunk with 7166 rows
Processing chunk with 5780 rows
Processing chunk with 6023 rows
Processing chunk with 5803 rows
Processing chunk with 7417 rows
Processing chunk with 6285 rows
Processing chunk with 6592 rows
Processi

In [ ]:
import ast
write_header = True
for chunk in works_df:
    chunk.drop(columns=['authorships'], inplace=True)
    print(f"Processing chunk with {len(chunk)} rows")
    chunk.to_csv('data/processed_works.csv',
                 mode='a', index=False, header=write_header)
    write_header = False

Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processing chunk with 1000 rows
Processi

In [100]:
import json
write_header = True
chunksize = 10000


def parse_json(json_str):
    try:
        json_str = json_str.replace("'", '"')
        # Handle single quotes in JSON string
        return json.loads(json_str)
    except json.JSONDecodeError:
        return None


works_df = pd.read_csv('data/works.csv', chunksize=chunksize,
                       converters={'open_access': parse_json})


# for chunk in works_df:
#     chunk['open_access'] = chunk['open_access'].apply(
#         lambda x: x.replace("'", '"'))
#     chunk.dropna(subset=['open_access'], inplace=True)
#     chunk['open_access'] = chunk['open_access'].apply(parse_json)
#     # Process the chunk further if needed
#     # For example, save the processed chunk to a new CSV file
#     chunk.to_csv('data/processed_works_open_access.csv',
#                  mode='a', index=False, header=write_header)
#     write_header = False
works_df.get_chunk(10)

,work_id,display_name,publication_date,open_access,cited_by_count,keywords
0,W4382515348,Penyuluhan dan Pelatihan Pembuatan Kompos Orga...,2023-06-29,None,1,[]
1,W4400300197,Peningkatan Pengetahuan Ibu Melalui Penyuluhan...,2023-11-01,None,0,[]
2,W4302328439,Polymorphism at codon 36 of the p53 gene.,1994-01-01,None,17,[{'id': 'https://openalex.org/keywords/single-...
3,W4391971651,Facile synthesis of KOH and ball milling co-mo...,2024-02-20,None,0,[{'id': 'https://openalex.org/keywords/methyle...
4,W4403955429,Facile Synthesis of Ball Milling and Koh Co-Mo...,2024-01-01,None,0,[{'id': 'https://openalex.org/keywords/methyle...
5,W3049151413,Réalisation d'un spectromètre de Fourier stati...,2009-05-29,None,0,[]
6,W2975492508,Cabo Verde Egyptian Vulture<i>Neophron percnop...,2019-09-23,None,2,[{'id': 'https://openalex.org/keywords/vulture...
7,W2915456352,RESPOSTAS ECOLÓGICAS DAS POPULAÇÕES DE CAMARÕE...,2018-11-11,None,0,[{'id': 'https://openalex.org/keywords/palaemo...
8,W2114182016,Gonadotrophins and ovarian steroids in cattle ...,1985-03-01,None,67,[{'id': 'https://openalex.org/keywords/luteoly...
9,W2947830255,P070 The effect of distance from specialist he...,2019-05-27,None,0,[]


In [101]:
chunk

,work_id,display_name,publication_date,open_access,cited_by_count,keywords
0,W4382515348,Penyuluhan dan Pelatihan Pembuatan Kompos Orga...,2023-06-29,None,1,[]
1,W4400300197,Peningkatan Pengetahuan Ibu Melalui Penyuluhan...,2023-11-01,None,0,[]
2,W4302328439,Polymorphism at codon 36 of the p53 gene.,1994-01-01,None,17,[{'id': 'https://openalex.org/keywords/single-...
3,W4391971651,Facile synthesis of KOH and ball milling co-mo...,2024-02-20,None,0,[{'id': 'https://openalex.org/keywords/methyle...
4,W4403955429,Facile Synthesis of Ball Milling and Koh Co-Mo...,2024-01-01,None,0,[{'id': 'https://openalex.org/keywords/methyle...
...,...,...,...,...,...,...
9995,W4401890391,Student Interpersonal Intelligence In The Age ...,2024-08-25,None,0,[{'id': 'https://openalex.org/keywords/publish...
9996,W4399811878,DAMPAK EKOWISATA KAWAH PUTIH DI KECAMATAN RANC...,2024-06-19,None,0,[]
9997,W4400267619,Laminated insulated glass units under blast lo...,2024-07-03,None,0,[{'id': 'https://openalex.org/keywords/laminat...
9998,W2079892328,Control and synchronizing nonlinear systems wi...,2013-09-04,None,2,[{'id': 'https://openalex.org/keywords/synchro...
